# Azure DevOps Release Pipelines

> *Azure DevOps Release Pipelines* are the bridge between your application development and its deployment to different environments. They are a series of automated steps and processes designed to facilitate the seamless deployment of your application to different target environments. 

## Release Pipelines and CI/CD

In a DevOps workflow, Continuous Integration is the first step in the software delivery pipeline. It focuses on automating the building, testing, and integration of code as soon as changes are committed to version control. We have seen in the previous lesson that **Azure DevOps Build Pipelines** are a core element of CI pipeline.

Continuous Deployment extends the CI process by automating the deployment of code to different environments. The CD pipeline takes over when CI confirms that the code is ready for production. Release Pipelines are the orchestrators of the deployment process. They coordinate the movement of code through various stages, aligning with the principles of CI/CD.

The benefits of automating the deployment process include:

- **Consistency**: When deployments are automated, you can rely on consistent processes and outcomes. Human error is minimized, and each deployment adheres to a predefined set of steps, resulting in uniform releases.

- **Speed**: Automation accelerates the delivery of software updates. It means your applications can reach your end-users faster, keeping them engaged with the latest features and bug fixes.

- **Reliability**: Automation doesn't just speed up the process; it makes it more reliable. Your deployment process becomes predictable, reducing the risk of unexpected failures and minimizing downtime.

- **Rollback Capability**: Automated deployment pipelines come with the safety net of rollback capabilities. If an issue arises during deployment, you can quickly revert to a previous, stable version, ensuring your users aren't affected.

## Azure DevOps Artifacts in Release Pipelines

*Artifacts* serve as building blocks for deploying your application to different environments. They play a pivotal role in the overall CI/CD process, acting as the intermediary between the build and deployment phases.

### Build Artifacts

*Build artifacts* are the most common type of artifacts used in release pipelines. These are the outcomes of the build process, generated when the code is compiled, tested, and packaged. Examples of build artifacts include:

- **Docker Images**: If you're containerizing your application, a Docker image is a typical build artifact. It contains your application and its dependencies, providing a consistent and isolated environment for deployment.

- **Compiled Code**: For applications that are not containerized, compiled code or binaries are common build artifacts. These are the executable files or libraries that can be deployed to target environments.

- **Installation Packages**: In scenarios where your application needs to be installed on a server, build artifacts may include installation packages that simplify the deployment process

In your release pipeline, you'll specify which build artifacts to use for deployment. These artifacts should be well-tested and ready for promotion to various stages, ensuring consistency and reliability in the deployment process. 

While they are powerful components of Azure DevOps, they are not mandatory to use in your Build/Release Pipelines. For example, you might have a Build Pipeline that builds a Docker image and then pushes it to Docker Hub. In this case, you won't need to create a build artifact, as the Release Pipeline can directly pull the image from Docker Hub instead, thus relying on an external artifact, which we will cover in the next section.

### Package Feeds

Azure DevOps provides package feeds to manage and distribute various types of packages, including libraries, frameworks, and modules. Package feeds ensure that dependencies and components your application relies on are readily available during deployment. By referencing these feeds, your release pipeline can retrieve and include the necessary packages in the deployment process. This reduces the risk of compatibility issues and ensures that the correct versions are deployed.

### External Artifacts

Sometimes, your application may rely on external sources or services that are not part of your Azure DevOps environment. External artifacts are references to these sources and are critical for managing deployments that involve external dependencies. Examples of external artifacts include:

- **Container Registries**: If your application relies on container images hosted in external container registries (e.g., Docker Hub), you would set up references to these external container repositories in your release pipeline

- **GitHub Repositories**: For source code, libraries, or other resources hosted on GitHub, you can configure references to GitHub repositories in your release pipeline

By specifying these external artifacts in your release pipeline, you ensure that the necessary external resources are accessible and integrated into your deployment process, maintaining a seamless and efficient workflow.

> In addition to artifacts, Release Pipelines include all the other components that Build Pipelines offer, ensuring a comprehensive approach to your CI/CD pipeline.

## Setting Up a Release Pipeline

In this section, we will walk you through the process of creating and configuring a Release Pipeline in Azure DevOps, focusing on deploying an example containerised application to an AKS cluster. We will build upon the example from the Build Pipeline lesson, so make sure to follow along that hands-on before.

> Remember, as mentioned in the previous lesson you won't have the necessary computer power access to run pipelines right now, but this will be provided to you when you reach the specialization project, where you will have to set up your own Azure DevOps Pipeline. So make sure to understand the provided example, but you don't need to replicate this example now.

### Step 1: Navigate to Azure DevOps and Access your Pipeline

1. Sign in with your Azure DevOps accounts credentials. After signing in, you'll be taken to the Azure DevOps dashboard.

2. To initiate the creation of a Release Pipeline, select the project where you have previously set up the Build Pipeline

In this hands-on guide, we will set up a Release Pipeline that deploys our example application to an AKS cluster. The repository of the application used here already contains the necessary Kubernetes deployment manifest, but not the `secrets.yaml` necessary to define the application environment variables. This file was uploaded to `.gitignore` because it contains sensitive information, and therefore Azure DevOps won't have access to the information stored there. Instead, we will add additional steps to this pipeline to automatically generate the environment variables needed for the Kubernetes manifest, and then apply these to the AKS cluster to deploy the sample application.

###  Step 2: Create Pipeline Variables

If the containerised application you want to deploy to AKS has sensitive information that is normally passed onto the Docker container during the `docker run` command as environment variables, or using a `secrets.yaml` for Kubernetes deployments, you will need to modify the pipeline configuration to include these environment variables. To achieve this you need to:
- Store your environment variables in Azure DevOps as pipeline variables
- Update the pipeline YAML to pass these environment variables to the Kubernetes deployment

To store environment variables in an Azure DevOps pipeline:
- Go to your Azure DevOps project, and navigate **Pipelines**, and click on **Library**

<p align=center> <img src=images/Library.png width=400 height=250> </p>

- Click on the **+ Variable group** button to create a new variable group. Name your variable group (e.g., flask-app-secrets) and add new variables for the environment variables that your Docker container requires. For example, if your Docker container has database authentication environment variables, add these here.

> Make sure to mark each variable as a secret if they contain sensitive information, like in the case of database authentication details. To do so, click on the lock simple on the right-hand side of the value field for that variable.

<p align=center> <img src=images/LibrarySecrets.png width=700 height=350> </p>


### Step 3: Update Pipeline YAML with Variables

To allow an Azure DevOps pipeline to access and utilise variables effectively, it's crucial to configure and manage these variables correctly within your pipeline definition. Variables in Azure DevOps pipelines can store values that are essential for various stages of your CI/CD processes, such as Docker builds, Kubernetes deployments, and more.

To use the variables defined in the variable groups, you need to edit the pipeline YAML file to use the `variables` section. This section is used to reference variable groups that you have created previously through Azure DevOps. For example, if you have defined a variable group named `flask-app-secrets`, this is how you would edit the YAML pipeline:

```yaml
variables:
  # Top-level variable groups definition
  - group: flask-app-secrets
```


### Step 4: Create and Apply Kubernetes Secrets

To create and apply Kubernetes secrets from pipeline variables, you will use the Azure CLI task in your Azure DevOps pipeline. This step ensures that sensitive information is securely managed and applied to your Kubernetes cluster.

First, we will use this command to install the Azure CLI: 

```bash
az aks install-cli
```

This command ensures that `kubectl` is installed on the build agent, allowing it to interact with your Kubernetes cluster.

Then, just as we did locally we can use the `az aks get-credentials --resource-group <resource-group-name> --name <aks-cluster-name>` to retrieve the credentials needed to access your AKS cluster and configure `kubectl` to use these credentials.

To create the secrets, we will a command similar to the one below:

```yaml
kubectl create secret generic <k8s-secret-name> \
  --from-literal=<secret-name>=$(<secret-name-in-variable-group) 
  --dry-run=client -o yaml > k8s-secret.yaml
```

This command creates a Kubernetes secret named `<k8s-secret-name>` using the values provided by the pipeline variables. The `--dry-run=client` flag ensures that the secret is generated in YAML format without actually applying it.

Finally, to apply the secrets, we will use the `kubectl apply -f k8s-secret.yaml` command, to apply the generated file to your AKS cluster, creating or updating the Kubernetes secret with the provided data.

Putting all the steps above together, the pipeline task would look like this:

```yaml
- task: AzureCLI@2
  inputs:
    azureSubscription: '<name of your aks-azure devops connection>'
    scriptType: 'bash'
    scriptLocation: 'inlineScript'
    inlineScript: |
      # Install kubectl
      az aks install-cli

      # Get AKS credentials
      az aks get-credentials --resource-group <resource-group-name> --name <aks-cluster-name>

      #verify that kubectl is correctly configured to interact with your AKS cluster 
      kubectl config current-context
      kubectl get nodes
      
      # Create Kubernetes secret
      kubectl create secret generic <name-of-secrets-group> \
        --from-literal=<secret-key-1>=$SECRET_VALUE1 \
        --from-literal=<secret-key-2>=$SECRET_VALUE2 \
        # add as many as necessary...
        --dry-run=client -o yaml > k8s-secret.yaml
      
      # Apply the Kubernetes secret
      kubectl apply -f k8s-secret.yaml
  displayName: 'Create and Apply Kubernetes Secrets'

```

In the example above, `secret-key1`: This is the name of the key within the Kubernetes Secret that you are creating. It should correspond to the key you used in your environment configuration or deployment manifest where the secret will be referenced. Make sure that this key name matches the expected key in your Kubernetes deployment configurations to ensure seamless integration. 

For the example above, if the secrets you are creating were referencing environment variables in your Kubernetes configuration, the  manifest would look as such:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-app
spec:
  replicas: 1
  selector:
    matchLabels:
      app: my-app
  template:
    metadata:
      labels:
        app: my-app
    spec:
      containers:
        - name: my-app-container
          image: my-docker-image:latest
          env:
            - name: SECRET_KEY_1
              valueFrom:
                secretKeyRef:
                  name: my-secrets-group
                  key: secret-key-1
            - name: SECRET_KEY_2
              valueFrom:
                secretKeyRef:
                  name: my-secrets-group
                  key: secret-key-2
```

The `$SECRET_VALUE1` represents the value associated with `secret-key1`. It is the actual secret data you want to store, which you previously defined in your Azure DevOps pipeline's variable group. `$SECRET_VALUE1` will be interpolated from the value of the secret retrieved from the pipeline library.

### Step 5: Apply the Kubernetes Deployment

In the pipeline task search bar, look for the **Deploy to Kubernetes** task. This task enables interaction with your AKS cluster using `kubectl` and provides the essential details for deployment. It's worth noting that alongside deployment-specific tasks, there's also a general `kubectl `task available. The `kubectl` task allows you to execute any type of `kubectl` commands, as we have seen in the previous step above.

In the **Deploy to Kubernetes** task, you'll need to specify the following information:

  - **Action**: Select the appropriate `kubectl` action, which is `deploy` in this case. This action tells the task what operation to perform on the Kubernetes cluster.

  - **Connection Type**: Choose **Azure Resource Manager** as the connection type. This option is selected because we are dealing with an AKS cluster, which is an Azure resource. To perform this step you should already have a Service Connection to AKS setup within your Azure DevOps project.

  - **Azure Subscription**: Here in the dropdown, you might see both the subscription and the name of the AKS-Azure DevOps connection you created in an earlier lesson. Make sure to select the AKS-Azure DevOps connection.

  - **Azure Resource Group**: Specify the Azure resource group where your AKS cluster is located. This defines the scope of the operation.

  - **Kubernetes Cluster**: Enter the name of your AKS cluster. This is the target cluster where your application will be deployed.

  - **Manifests Path**: Provide the path to the Kubernetes deployment manifests that describe your application. These manifests contain configuration details for your application's deployment, such as deployment, service, and ingress resources. If the manifest is located in the same directory as the CI/CD pipeline (e.g. root directory of the repository), you can simply reference the manifest file by its name here. However, if the manifest file is in a different location, make sure to provide the correct path followed by the name of the manifest file. 

<p align=center> <img src=images/DeploymentTask.png width=800 height=300> </p>

### Step 6: Running and Verifying Your Azure DevOps Build Pipeline

After configuring your Azure DevOps Release Pipeline to deploy your application to an AKS cluster, it's essential to run the pipeline and ensure that it performs as expected.

To initiate the deployment process manually, click the **Run** button. A **Run Pipeline** window will appear. The **Branch** dropdown menu allows you to select the branch from which the pipeline will be triggered. By default, it might show the branch specified in your pipeline trigger configuration (e.g., `main`).

You can also specify a commit hash to trigger the pipeline from a specific commit in the selected branch. This can be useful when you want to re-run the pipeline for a particular commit. Here, we will leave this empty.

After configuring the options as needed, click the **Run** button to start the pipeline. The pipeline execution will start, and you can monitor its progress in real-time. 

The tasks you've defined in the pipeline, such as building the Docker image and pushing it to Docker Hub, will be executed in sequence. You can monitor these tasks, by clicking on the **Job** button.

> If you are using variables group within the pipeline configuration, the first time you run the pipeline you will need to give permissions to access the variable group, before the job can continue running.

<p align=center> <img src=images/MonitorJobRun.png width=800 height=400> </p>

You will be redirected to a window, where you will a list of jobs that are part of your pipeline. Each job represents a set of tasks to be executed.

<p align=center> <img src=images/JobSteps.png width=800 height=400> </p>

Once the pipeline run is completed, review the pipeline logs, which can be found in the **Jobs** section to check for any errors or issues during the build and push tasks. If the job was successful you should see the job and all its associated steps green ticked, indicating each step was successful.

<p align=center> <img src=images/SuccessfulJob.png width=800 height=400> </p>


### Step 7: Check Application Deployment on AKS

Once the job has been successfully executed, it's important to verify whether the application has been correctly deployed to the AKS cluster and is running as expected. 

Use the `kubectl get pods` command to monitor the status of your pods. Initially, you might observe the status as `ContainerCreating`, but it should transition to `Running` over time. The duration of this transition may vary depending on your application's complexity.

<p align=center> <img src=images/KubectlPods.png width=500 height=350> </p>

To gain insight into your application's behavior and ensure it's running correctly, you can access the logs of a specific pod. Use the `kubectl logs` command followed by the pod name: `kubectl logs -f <pod name>`.

<p align=center> <img src=images/KubectLogs.png width=600 height=300> </p>

The log output should provide information about the application's activities. For instance, you might see messages that you included in your application's code. For the example application used above, we can see the following message: `"Acquiring book list"`, which was part of the application code, indicating the application is working as expected.

By following these steps, you can successfully configure and execute a Release Pipeline that automates the deployment of any application to an AKS cluster. Here we have looked at an example application, but the steps are extensible to any, you just need to make sure you have the correct Kubernetes manifests. You've also learned how to verify the deployment and monitor the application's behavior using `kubectl` commands, ensuring a reliable and efficient deployment process.

## Key Takeaways

- Release Pipelines in Azure DevOps serve as a crucial component in the Continuous Integration and Continuous Deployment (CI/CD) process, facilitating the automated deployment of your applications to various target environments
- Artifacts are the deployable units in Release Pipelines. They can be built from a build pipeline, sourced from external locations, or come from package feeds. Artifacts can form the basis for your deployment process.
- **Deploy to Kubernetes** is a task that allows you to deploy applications to an Azure Kubernetes Service (AKS) cluster using Kubernetes command-line (`kubectl`) actions. It involves specifying essential details like the Azure subscription, resource group, Kubernetes cluster, and manifest files.
- Verification and monitoring are critical steps in the deployment process. You can use `kubectl` commands to check the status of your pods and review application logs to ensure your application is running correctly.